In [17]:
import pandas as pd
import json

def merge(filename:str,station_name:pd.DataFrame) -> pd.DataFrame:
    df1 = pd.read_csv(filename)
    df2 = df1.rename(columns={'trnOpDate':'日期',
                    'staCode':'車站代碼',
                    'gateInComingCnt':'進站人數',
                    'gateOutGoingCnt':'出站人數'})
    df2['日期'] = pd.to_datetime(df2['日期'].astype(str))
    result_df = pd.merge(df2,station_name,left_on='車站代碼',right_on='編碼',how='left')
    return result_df

with open('每日各站進出站人數/車站基本資料集.json',encoding='utf-8') as file:
    json_data:list[dict] = json.load(file)

stations_name = pd.DataFrame(json_data,columns=['stationCode','stationName','stationAddrTw'])
stations_name1 = stations_name.rename(columns={'stationCode':'編碼',
                      'stationName':'車站名稱',
                      'stationAddrTw':'地址'})
stations_name1['編碼'] = stations_name1['編碼'].astype(int)



In [18]:
import os
current_path = os.path.abspath(__name__) #目前執行檔案的絕對路徑
main_path = os.path.dirname(current_path) #取得目前執行的目錄的絕對路徑

'每日各站進出站人數'
dataFolder_path = os.path.join(main_path,'每日各站進出站人數') #取得放資料的目錄
filenames = os.listdir(dataFolder_path) #取得資料目錄的檔案名稱
names = list(filter(lambda name: '每日各站進出站人數' in name,filenames)) #lambda
abs_names = [os.path.join(dataFolder_path,name) for name in names] #list comprehension
all_data = [merge(filename=abs_n,station_name=stations_name1) for abs_n in abs_names] #list comprehension
all_data
    



[              日期  車站代碼  進站人數  出站人數    編碼 車站名稱  \
 0     2020-01-01   900  7552  8154   900   基隆   
 1     2020-01-01   910  1020  1135   910   三坑   
 2     2020-01-01   920  1623  1972   920   八堵   
 3     2020-01-01   930  4130  4813   930   七堵   
 4     2020-01-01   940  1818  2128   940   百福   
 ...          ...   ...   ...   ...   ...  ...   
 86927 2020-12-31  7360  3523  3602  7360   瑞芳   
 86928 2020-12-31  7361     3    18  7361  海科館   
 86929 2020-12-31  7362    28    34  7362  八斗子   
 86930 2020-12-31  7380   609   604  7380  四腳亭   
 86931 2020-12-31  7390   377   444  7390   暖暖   
 
                                                   地址  
 0                                        基隆市仁愛區港西街5號  
 1                                 基隆市仁愛區德厚里龍安街 206 號  
 2                                 基隆市暖暖區八南里八堵路 142 號  
 3                                   基隆市七堵區長興里東新街 2 號  
 4                              基隆市七堵區堵南里明德三路 1 之 1 號  
 ...                                              ...  
 86927